In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from finvizfinance.screener.overview import Overview
from scipy.stats import linregress
import time

In [ ]:
# ==========================================
# 1.(Finviz Screening)
# ==========================================
def get_filtered_picks(limit_per_country=5000): # Raised limit to capture more
    print("--- STEP 1: Fetching Universe from Finviz ---")
    
    try:
        foverview = Overview()
        
        # Criteria: USA, Strong Buy, High Liquidity, Not Penny Stocks
        filters_dict = {
            
            'Country': 'USA',
            'Average Volume': 'Over 2M', 
            'Market Cap.': '+Small (over $300mln)',
            # 1. Institutional Floor (Removes Penny Stocks)
            'Price': 'Over $5',
    
            # 2. Legitimacy Check (Ensures Options Market exists)
            'Option/Short': 'Optionable',
    
            # 3. Solvency Check (Basic Liquidity) consider removing since it filters out telecom and utilities companies
            'Current Ratio': 'Over 1',
    
            # 4. Sector Exclusion (If you want to remove volatility) thinking about using later
            # 'Industry': 'Stocks only (ex-Funds)',

            #5 profitability filter
            'Operating Margin': 'Positive (>0%)',
            
            #6 Analyst Rating Filter consider changing later
            'Analyst Recom.': 'Strong Buy (1)',
        }
        foverview.set_filter(filters_dict=filters_dict)
        df_results = foverview.screener_view()
        
        if df_results.empty:
            return pd.DataFrame()
            
        print(f"   Success! Found {len(df_results)} initial candidates.")
        
        # Clean up Column Names immediately
        if 'Analyst Recom' in df_results.columns:
            df_results.rename(columns={'Analyst Recom': 'Recom'}, inplace=True)
            
        # Ensure numeric Price for later
        df_results['Price'] = pd.to_numeric(df_results['Price'], errors='coerce')
        
        return df_results

    except Exception as e:
        print(f"   Error in Finviz Step: {e}")
        return pd.DataFrame()

In [ ]:
# ==========================================
# 2. THE CREDIT MODEL (Z-Score & Margins)
# ==========================================
def calculate_z_score(info, financials, balance_sheet):
    """Calculates Altman Z-Score (Bankruptcy Risk)"""
    try:
        # Extract Key Metrics
        total_assets = balance_sheet.loc['Total Assets'].iloc[0]
        total_liab = balance_sheet.loc['Total Liabilities Net Minority Interest'].iloc[0]
        current_assets = balance_sheet.loc['Current Assets'].iloc[0]
        current_liab = balance_sheet.loc['Current Liabilities'].iloc[0]
        
        # Z-Score Components
        working_capital = current_assets - current_liab
        retained_earnings = balance_sheet.loc['Retained Earnings'].iloc[0] if 'Retained Earnings' in balance_sheet.index else 0
        
        # Handle EBIT naming differences
        if 'Ebit' in financials.index:
            ebit = financials.loc['Ebit'].iloc[0]
        elif 'Operating Income' in financials.index:
            ebit = financials.loc['Operating Income'].iloc[0]
        else:
            return np.nan
            
        market_cap = info.get('marketCap', 0)
        sales = financials.loc['Total Revenue'].iloc[0]

        # Ratios
        A = working_capital / total_assets
        B = retained_earnings / total_assets
        C = ebit / total_assets
        D = market_cap / total_liab
        E = sales / total_assets

        # Formula: 1.2A + 1.4B + 3.3C + 0.6D + 1.0E Altman Z-Score coefficients
        z_score = (1.2 * A) + (1.4 * B) + (3.3 * C) + (0.6 * D) + (1.0 * E)
        return round(z_score, 2)
    except:
        return np.nan
#The Margin Trend column is a custom metric we built to measure operational momentum. 
# It tells you whether a company is becoming more or less efficient at making money over time.
def get_margin_trend(financials):
    """Calculates 3-Year Gross Margin Slope"""
    try:
        years = financials.columns[:3]
        margins = []
        for date in years:
            rev = financials.loc['Total Revenue'][date]
            profit = financials.loc['Gross Profit'][date]
            if rev == 0 or np.isnan(rev): margins.append(0)
            else: margins.append(profit/rev)
            
        # Slope Calculation
        margins = margins[::-1] # Chronological order
        slope, _, _, _, _ = linregress(range(len(margins)), margins)
        
        if slope > 0.005: return "Improving"
        elif slope < -0.005: return "Deteriorating"
        else: return "Stable"
    except:
        return "N/A"


In [ ]:
# ==========================================
# 3. EXECUTION ENGINE (SPLIT BY TIER)
# ==========================================
import time

def run_screener_split():
    # A. Get the List from Step 1
    candidates = get_filtered_picks()
    
    if candidates.empty:
        print("No stocks found to analyze.")
        return None, None, None, None, None

    print(f"\n--- STEP 2: Running Credit Risk Model on {len(candidates)} Stocks ---")
    print("Separating stocks into distinct tiers...")
    
    # Initialize separate lists for each tier
    fortress_data = []
    moonshot_data = []
    distress_data = []
    financial_data = [] # Banks/Insurance (skipped by Z-score)
    middle_data = []    # Everything else
    
    # Loop through ALL candidates
    for index, row in candidates.iterrows():
        ticker = row['Ticker']
        
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            
            # 1. SEPARATE FINANCIALS
            # Z-Score doesn't work for banks, so we stash them here to analyze later
            sector = info.get('sector', 'Unknown')
            if 'Financial' in sector:
                financial_data.append({
                    'Ticker': ticker,
                    'Sector': sector,
                    'Price': row['Price'],
                    'Recom': row.get('Recom', 'N/A')
                })
                continue
            
            # 2. FETCH DATA
            fin = stock.financials
            bs = stock.balance_sheet
            
            if fin.empty or bs.empty:
                continue
            
            # 3. RUN METRICS (Using your helper functions)
            z_score = calculate_z_score(info, fin, bs)
            trend = get_margin_trend(fin)
            
            # Data packet to save
            stock_data = {
                'Ticker': ticker,
                'Company': row.get('Company', 'N/A'),
                'Z-Score': z_score,
                'Margin_Trend': trend, 
                'Price': row['Price'],
                'Recom': row.get('Recom', 'N/A'),
                'Sector': sector
            }
            
            # 4. SORT INTO LISTS (The Logic)
            if (z_score > 2.99) and (trend in ["Improving", "Stable"]):
                fortress_data.append(stock_data)
                
            elif (z_score < 1.8) and (trend == "Deteriorating"):
                distress_data.append(stock_data)
                
            elif (z_score < 1.8) and (trend == "Improving"):
                moonshot_data.append(stock_data)
            
            else:
                middle_data.append(stock_data)
            
            # Progress indicator
            if index % 10 == 0:
                print(f"   Processed {index} / {len(candidates)}...", end='\r')

        except Exception:
            continue

    # B. Convert Lists to DataFrames
    fortress_df = pd.DataFrame(fortress_data)
    moonshot_df = pd.DataFrame(moonshot_data)
    distress_df = pd.DataFrame(distress_data)
    financial_df = pd.DataFrame(financial_data)
    middle_df = pd.DataFrame(middle_data)
    
    print(f"\n\n--- ANALYSIS COMPLETE ---")
    print(f"Fortress (Safe):     {len(fortress_df)} stocks")
    print(f"Moonshot (Risky):    {len(moonshot_df)} stocks")
    print(f"Distress (Avoid):    {len(distress_df)} stocks")
    print(f"Financials (Skip):   {len(financial_df)} stocks")
    print(f"Middle of Road:      {len(middle_df)} stocks")
    
    return fortress_df, moonshot_df, distress_df, financial_df, middle_df

# --- RUN IT & UNPACK VARIABLES ---
# This saves the results into 5 separate dataframes immediately
fortress_df, moonshot_df, distress_df, fins_df, mid_df = run_screener_split()
#fins_df are banking and other financial sector stocks cannot use Z score due to customer deposits being liabilities

--- STEP 1: Fetching Universe from Finviz ---
   Success! Found 551 initial candidates.#########-] 27/28 

--- STEP 2: Running Credit Risk Model on 551 Stocks ---
Separating stocks into distinct tiers...
   Processed 550 / 551...

--- ANALYSIS COMPLETE ---
Fortress (Safe):     171 stocks
Moonshot (Risky):    54 stocks
Distress (Avoid):    56 stocks
Financials (Skip):   51 stocks
Middle of Road:      217 stocks


In [5]:
if fortress_df is not None and not fortress_df.empty:
    print("--- FORTRESS STOCKS (High Quality) ---")
    
    # 1. NEW: Filter for Price > $5 (Institutional Standard)
    # We create a copy to avoid SettingWithCopy warnings
    filtered_fortress = fortress_df[ fortress_df['Price'] > 5.0 ].copy()
    
    if not filtered_fortress.empty:
        print(f"Showing {len(filtered_fortress)} stocks priced over $5.")
        
        # 2. Sort by Z-Score (Healthiest First)
        display(filtered_fortress.sort_values(by='Z-Score', ascending=False))
        
    else:
        print("No Fortress stocks found over $5.")

--- FORTRESS STOCKS (High Quality) ---
Showing 171 stocks priced over $5.


,Ticker,Company,Z-Score,Margin_Trend,Price,Recom,Sector
119,PPTA,Perpetua Resources Corp,223.80,Stable,27.60,N/A,Basic Materials
117,PLTR,Palantir Technologies Inc,222.26,Improving,193.38,N/A,Technology
124,QXO,QXO Inc,195.61,Stable,21.80,N/A,Industrials
7,ALAB,Astera Labs Inc,186.51,Improving,164.40,N/A,Technology
161,UMAC,Unusual Machines Inc,165.18,Improving,9.85,N/A,Technology
...,...,...,...,...,...,...,...
154,TSN,"Tyson Foods, Inc",3.11,Improving,58.47,N/A,Consumer Defensive
14,APG,APi Group Corporation,3.07,Improving,38.88,N/A,Industrials
136,SLB,SLB Ltd,3.06,Improving,38.08,N/A,Energy
113,PAYX,Paychex Inc,3.06,Improving,112.28,N/A,Technology


In [6]:
#high risk picks
if moonshot_df is not None and not moonshot_df.empty:
    print("--- MOONSHOT STOCKS (Speculative) ---")
    # Sort by Price (Cheapest first?) or Recom
    display(moonshot_df.sort_values(by='Price', ascending=True))

--- MOONSHOT STOCKS (Speculative) ---


,Ticker,Company,Z-Score,Margin_Trend,Price,Recom,Sector
1,AQST,Aquestive Therapeutics Inc,-2.00,Improving,5.80,N/A,Healthcare
29,MNKD,Mannkind Corp,-7.24,Improving,5.84,N/A,Healthcare
49,UDMY,Udemy Inc,0.54,Improving,5.97,N/A,Consumer Defensive
40,PTON,Peloton Interactive Inc,-1.45,Improving,6.20,N/A,Consumer Cyclical
48,TE,T1 Energy Inc,-0.04,Improving,6.43,N/A,Industrials
36,NVAX,"Novavax, Inc",-4.31,Improving,6.66,N/A,Healthcare
44,SG,Sweetgreen Inc,0.39,Improving,6.85,N/A,Consumer Cyclical
47,TDOC,Teladoc Health Inc,-5.30,Improving,7.23,N/A,Healthcare
53,XERS,Xeris Biopharma Holdings Inc,-0.31,Improving,7.34,N/A,Healthcare
0,ACVA,ACV Auctions Inc,1.47,Improving,7.80,N/A,Consumer Cyclical


In [7]:
#very high risk / avoid picks
if distress_df is not None and not distress_df.empty:
    print("--- DISTRESS (Avoid / Short Candidates) ---")
    display(distress_df.sort_values(by='Z-Score', ascending=True))

--- DISTRESS (Avoid / Short Candidates) ---


,Ticker,Company,Z-Score,Margin_Trend,Price,Recom,Sector
50,VERI,Veritone Inc,-2.82,Deteriorating,5.01,N/A,Technology
21,FLY,Firefly Aerospace Inc,-2.16,Deteriorating,24.65,N/A,Industrials
54,WOLF,Wolfspeed Inc,-1.88,Deteriorating,18.37,N/A,Technology
20,FCEL,Fuelcell Energy Inc,-1.33,Deteriorating,8.34,N/A,Industrials
5,APLS,Apellis Pharmaceuticals Inc,-0.87,Deteriorating,24.65,N/A,Healthcare
46,SAIL,Sailpoint Inc,-0.49,Deteriorating,21.02,N/A,Technology
11,BULL,Webull Corp,-0.47,Deteriorating,8.57,N/A,Technology
6,APPS,Digital Turbine Inc,-0.23,Deteriorating,5.26,N/A,Technology
45,RUM,Rumble Inc,-0.23,Deteriorating,6.90,N/A,Communication Services
31,LUMN,Lumen Technologies Inc,-0.13,Deteriorating,8.19,N/A,Communication Services
